In [1]:
import pandas as pd
from datetime import datetime, timedelta
from isodate import duration_isoformat
import warnings

warnings.filterwarnings("ignore")

In [2]:
from api_test import use_token, get_path, calculate_co2_emissions
from utils import fetch_stations, get_home_paths, get_work_path

In [3]:
LIMIT = 5

start_lat = "46.523904"
start_lon = "6.564732"

target_lat = "46.505265"
target_lon = "6.627317"

date = "2023-12-02"
time = "17:07"

In [4]:
start_stations = pd.DataFrame(fetch_stations(start_lat, start_lon, limit=LIMIT))
target_stations = pd.DataFrame(fetch_stations(target_lat, target_lon, limit=LIMIT))

df = pd.merge(
    start_stations, target_stations, how="cross", suffixes=["_start", "_target"]
)

In [5]:
df

,name_start,id_start,coordinates_start,distance_start,name_target,id_target,coordinates_target,distance_target
0,"Ecublens VD, Bassenges",8501215,"[6.564588, 46.524588]",76,Lausanne-Ouchy (lac),8501075,"[6.627612, 46.505082]",30
1,"Ecublens VD, Bassenges",8501215,"[6.564588, 46.524588]",76,"Lausanne, Ouchy-Olympique",8592086,"[6.626641, 46.507473]",251
2,"Ecublens VD, Bassenges",8501215,"[6.564588, 46.524588]",76,"Lausanne, Beau-Rivage",8591986,"[6.627504, 46.5083]",338
3,"Ecublens VD, Bassenges",8501215,"[6.564588, 46.524588]",76,"Lausanne, Parc Musée Olympique",8595933,"[6.632898, 46.507338]",485
4,"Ecublens VD, Bassenges",8501215,"[6.564588, 46.524588]",76,"Lausanne, Jordils",8592061,"[6.627432, 46.509927]",519
5,"Ecublens VD, EPFL",8501214,"[6.566144, 46.522197]",218,Lausanne-Ouchy (lac),8501075,"[6.627612, 46.505082]",30
6,"Ecublens VD, EPFL",8501214,"[6.566144, 46.522197]",218,"Lausanne, Ouchy-Olympique",8592086,"[6.626641, 46.507473]",251
7,"Ecublens VD, EPFL",8501214,"[6.566144, 46.522197]",218,"Lausanne, Beau-Rivage",8591986,"[6.627504, 46.5083]",338
8,"Ecublens VD, EPFL",8501214,"[6.566144, 46.522197]",218,"Lausanne, Parc Musée Olympique",8595933,"[6.632898, 46.507338]",485
9,"Ecublens VD, EPFL",8501214,"[6.566144, 46.522197]",218,"Lausanne, Jordils",8592061,"[6.627432, 46.509927]",519


In [6]:
df["home_to_station_path"] = [
    get_home_paths(
        [start_lon, start_lat],
        df.loc[i, "coordinates_start"],
        df.loc[i, "distance_start"],
        date,
        time,
        df.loc[i, "id_start"],
    )
    for i in range(len(df))
]

In [7]:
df = df.explode("home_to_station_path").reset_index(drop=True)

In [8]:
headers = use_token()


def get_reduced_path(origin: str, destination: str, date: str, time: str):
    results = get_path(headers, origin, destination, date, time)
    return [(result["path"], result["co2_emissions"]) for result in results]

In [9]:
df["station_to_station_path"] = [
    get_reduced_path(
        df.loc[i, "id_start"],
        df.loc[i, "id_target"],
        df.loc[i, "home_to_station_path"]["arrival_date"],
        df.loc[i, "home_to_station_path"]["arrival_time"],
    )
    for i in range(len(df))
]

In [10]:
paths = df.explode("station_to_station_path").reset_index(drop=True)

In [11]:
paths["CO2 emissions"] = [x[1] for x in paths["station_to_station_path"]]
paths["station_to_station_path"] = [x[0] for x in paths["station_to_station_path"]]

In [12]:
paths["station_to_work_path"] = [
    [
        get_work_path(
            paths.loc[i, "coordinates_start"],
            paths.loc[i, "coordinates_target"],
            paths.loc[i, "distance_start"],
            "FOOT",
            paths.loc[i, "station_to_station_path"][-1]["arrival_date"],
            paths.loc[0, "station_to_station_path"][-1]["arrival_time"],
        ),
        get_work_path(
            paths.loc[i, "coordinates_start"],
            paths.loc[i, "coordinates_target"],
            paths.loc[i, "distance_start"],
            "BIKE",
            paths.loc[i, "station_to_station_path"][-1]["arrival_date"],
            paths.loc[0, "station_to_station_path"][-1]["arrival_time"],
        ),
        get_work_path(
            paths.loc[i, "coordinates_start"],
            paths.loc[i, "coordinates_target"],
            paths.loc[i, "distance_start"],
            "CAR",
            paths.loc[i, "station_to_station_path"][-1]["arrival_date"],
            paths.loc[0, "station_to_station_path"][-1]["arrival_time"],
        ),
    ]
    for i in range(len(paths))
]

In [13]:
paths = paths.explode("station_to_work_path").reset_index(drop=True)

In [14]:
paths["CO2 emissions"] = [
    paths.loc[i, "CO2 emissions"]
    + calculate_co2_emissions(paths.loc[i, "home_to_station_path"])
    + calculate_co2_emissions(paths.loc[i, "station_to_work_path"])
    for i in range(len(paths))
]

In [15]:
paths

,name_start,id_start,coordinates_start,distance_start,name_target,id_target,coordinates_target,distance_target,home_to_station_path,station_to_station_path,CO2 emissions,station_to_work_path
0,"Ecublens VD, Bassenges",8501215,"[6.564588, 46.524588]",76,Lausanne-Ouchy (lac),8501075,"[6.627612, 46.505082]",30,"{'mode': 'FOOT', 'duration': 'PT54.72S', 'depa...","[{'mode': 'METRO', 'duration': 'PT15M', 'depar...",0.289003,"{'mode': 'FOOT', 'duration': 'PT54.72S', 'depa..."
1,"Ecublens VD, Bassenges",8501215,"[6.564588, 46.524588]",76,Lausanne-Ouchy (lac),8501075,"[6.627612, 46.505082]",30,"{'mode': 'FOOT', 'duration': 'PT54.72S', 'depa...","[{'mode': 'METRO', 'duration': 'PT15M', 'depar...",0.289003,"{'mode': 'BIKE', 'duration': 'PT13.68S', 'depa..."
2,"Ecublens VD, Bassenges",8501215,"[6.564588, 46.524588]",76,Lausanne-Ouchy (lac),8501075,"[6.627612, 46.505082]",30,"{'mode': 'FOOT', 'duration': 'PT54.72S', 'depa...","[{'mode': 'METRO', 'duration': 'PT15M', 'depar...",1.857629,"{'mode': 'CAR', 'duration': 'PT4.56S', 'depart..."
3,"Ecublens VD, Bassenges",8501215,"[6.564588, 46.524588]",76,Lausanne-Ouchy (lac),8501075,"[6.627612, 46.505082]",30,"{'mode': 'FOOT', 'duration': 'PT54.72S', 'depa...","[{'mode': 'METRO', 'duration': 'PT5M', 'depart...",0.303140,"{'mode': 'FOOT', 'duration': 'PT54.72S', 'depa..."
4,"Ecublens VD, Bassenges",8501215,"[6.564588, 46.524588]",76,Lausanne-Ouchy (lac),8501075,"[6.627612, 46.505082]",30,"{'mode': 'FOOT', 'duration': 'PT54.72S', 'depa...","[{'mode': 'METRO', 'duration': 'PT5M', 'depart...",0.303140,"{'mode': 'BIKE', 'duration': 'PT13.68S', 'depa..."
...,...,...,...,...,...,...,...,...,...,...,...,...
715,"Ecublens VD, Motty",8591951,"[6.561568, 46.525874]",326,"Lausanne, Jordils",8592061,"[6.627432, 46.509927]",519,"{'mode': 'BIKE', 'duration': 'PT58.68S', 'depa...","[{'mode': 'BUS', 'duration': 'PT3M', 'departur...",0.302043,"{'mode': 'BIKE', 'duration': 'PT58.68S', 'depa..."
716,"Ecublens VD, Motty",8591951,"[6.561568, 46.525874]",326,"Lausanne, Jordils",8592061,"[6.627432, 46.509927]",519,"{'mode': 'BIKE', 'duration': 'PT58.68S', 'depa...","[{'mode': 'BUS', 'duration': 'PT3M', 'departur...",1.913314,"{'mode': 'CAR', 'duration': 'PT19.56S', 'depar..."
717,"Ecublens VD, Motty",8591951,"[6.561568, 46.525874]",326,"Lausanne, Jordils",8592061,"[6.627432, 46.509927]",519,"{'mode': 'BIKE', 'duration': 'PT58.68S', 'depa...","[{'mode': 'BUS', 'duration': 'PT3M', 'departur...",0.302043,"{'mode': 'FOOT', 'duration': 'PT3M54.72S', 'de..."
718,"Ecublens VD, Motty",8591951,"[6.561568, 46.525874]",326,"Lausanne, Jordils",8592061,"[6.627432, 46.509927]",519,"{'mode': 'BIKE', 'duration': 'PT58.68S', 'depa...","[{'mode': 'BUS', 'duration': 'PT3M', 'departur...",0.302043,"{'mode': 'BIKE', 'duration': 'PT58.68S', 'depa..."


In [16]:
final_df = pd.DataFrame(
    {
        "path": [
            [paths.loc[i, "home_to_station_path"]]
            + paths.loc[i, "station_to_station_path"]
            + [paths.loc[i, "station_to_work_path"]]
            for i in range(len(paths))
        ],
        "CO2 emissions": paths["CO2 emissions"],
    }
)

In [17]:
final_df

,path,CO2 emissions
0,"[{'mode': 'FOOT', 'duration': 'PT54.72S', 'dep...",0.289003
1,"[{'mode': 'FOOT', 'duration': 'PT54.72S', 'dep...",0.289003
2,"[{'mode': 'FOOT', 'duration': 'PT54.72S', 'dep...",1.857629
3,"[{'mode': 'FOOT', 'duration': 'PT54.72S', 'dep...",0.303140
4,"[{'mode': 'FOOT', 'duration': 'PT54.72S', 'dep...",0.303140
...,...,...
715,"[{'mode': 'BIKE', 'duration': 'PT58.68S', 'dep...",0.302043
716,"[{'mode': 'BIKE', 'duration': 'PT58.68S', 'dep...",1.913314
717,"[{'mode': 'BIKE', 'duration': 'PT58.68S', 'dep...",0.302043
718,"[{'mode': 'BIKE', 'duration': 'PT58.68S', 'dep...",0.302043


In [18]:
final_df["time"] = 0
final_df["transfers"] = 0

In [19]:
def transform(row):
    paths = row["path"]

    if paths[0]["arrival_time"] != paths[1]["departure_time"]:
        diff = datetime.strptime(
            f"{paths[1]['departure_date']} {paths[1]['departure_time']}",
            "%Y-%m-%d %H:%M",
        ) - datetime.strptime(
            f"{paths[0]['arrival_date']} {paths[0]['arrival_time']}", "%Y-%m-%d %H:%M"
        )
        paths[0]["departure_date"] = (
            datetime.strptime(
                f"{paths[0]['departure_date']} {paths[0]['departure_time']}",
                "%Y-%m-%d %H:%M",
            )
            + diff
        ).strftime("%Y-%m-%d")
        paths[0]["departure_time"] = (
            datetime.strptime(
                f"{paths[0]['departure_date']} {paths[0]['departure_time']}",
                "%Y-%m-%d %H:%M",
            )
            + diff
        ).strftime("%H:%M")
        paths[0]["arrival_date"] = (
            datetime.strptime(
                f"{paths[0]['arrival_date']} {paths[0]['arrival_time']}",
                "%Y-%m-%d %H:%M",
            )
            + diff
        ).strftime("%Y-%m-%d")
        paths[0]["arrival_time"] = (
            datetime.strptime(
                f"{paths[0]['arrival_date']} {paths[0]['arrival_time']}",
                "%Y-%m-%d %H:%M",
            )
            + diff
        ).strftime("%H:%M")

    time = (
        datetime.strptime(
            f"{paths[-1]['arrival_date']} {paths[-1]['arrival_time']}",
            "%Y-%m-%d %H:%M",
        )
        - datetime.strptime(
            f"{paths[0]['departure_date']} {paths[0]['departure_time']}",
            "%Y-%m-%d %H:%M",
        )
    ).seconds

    transfers = len(paths)

    row["time"] = time
    row["transfers"] = transfers

    return row

In [20]:
final_df = final_df.apply(transform, axis=1)

In [21]:
def normalize_column(column):
    min_val = float(column.min())
    max_val = float(column.max())
    normalized_column = (column - min_val) / (max_val - min_val)
    return normalized_column

In [22]:
final_df["CO2 emissions"] = normalize_column(final_df["CO2 emissions"])
final_df["time"] = normalize_column(final_df["time"])
final_df["transfers"] = normalize_column(final_df["transfers"])

In [23]:
final_df

,path,CO2 emissions,time,transfers
0,"[{'mode': 'FOOT', 'duration': 'PT54.72S', 'dep...",0.015501,0.018081,0.0
1,"[{'mode': 'FOOT', 'duration': 'PT54.72S', 'dep...",0.015501,0.018081,0.0
2,"[{'mode': 'FOOT', 'duration': 'PT54.72S', 'dep...",0.877925,0.018081,0.0
3,"[{'mode': 'FOOT', 'duration': 'PT54.72S', 'dep...",0.023274,0.015994,1.0
4,"[{'mode': 'FOOT', 'duration': 'PT54.72S', 'dep...",0.023274,0.015994,1.0
...,...,...,...,...
715,"[{'mode': 'BIKE', 'duration': 'PT58.68S', 'dep...",0.022671,0.002086,1.0
716,"[{'mode': 'BIKE', 'duration': 'PT58.68S', 'dep...",0.908540,0.002086,1.0
717,"[{'mode': 'BIKE', 'duration': 'PT58.68S', 'dep...",0.022671,0.984701,1.0
718,"[{'mode': 'BIKE', 'duration': 'PT58.68S', 'dep...",0.022671,0.982615,1.0


In [24]:
final_df["score"] = 0

In [25]:
def calculate_score(row, co2_weight, time_weight, transfers_weight):
    return (
        co2_weight * row["CO2 emissions"]
        + time_weight * row["time"]
        + transfers_weight * row["transfers"]
    )

In [26]:
calculate_score(final_df.iloc[0, :], 0.5, 0.9, 0.7)

0.02402308871853746

In [27]:
calculate_score(final_df.iloc[716, :], 0.5, 0.9, 0.7)

1.1561476336497272

In [28]:
calculate_score(final_df.iloc[719, :], 0.5, 0.9, 0.7)

2.038623294289505

In [29]:
co2_weight, time_weight, transfers_weight = 0.5, 0.9, 0.7

In [30]:
final_df["score"] = final_df.apply(
    lambda x: calculate_score(x, co2_weight, time_weight, transfers_weight), axis=1
)

In [31]:
final_df.sort_values(by=["score"])

,path,CO2 emissions,time,transfers,score
457,"[{'mode': 'BIKE', 'duration': 'PT47.34S', 'dep...",0.000000,0.002782,0.0,0.002503
289,"[{'mode': 'BIKE', 'duration': 'PT39.24S', 'dep...",0.000000,0.002782,0.0,0.002503
493,"[{'mode': 'BIKE', 'duration': 'PT47.34S', 'dep...",0.000000,0.002782,0.0,0.002503
325,"[{'mode': 'BIKE', 'duration': 'PT39.24S', 'dep...",0.000000,0.002782,0.0,0.002503
573,"[{'mode': 'FOOT', 'duration': 'PT3M9.36S', 'de...",0.007012,0.000000,0.0,0.003506
...,...,...,...,...,...
605,"[{'mode': 'FOOT', 'duration': 'PT3M52.56S', 'd...",0.938315,0.984006,1.0,2.054762
707,"[{'mode': 'BIKE', 'duration': 'PT58.68S', 'dep...",0.993120,0.982615,1.0,2.080913
701,"[{'mode': 'FOOT', 'duration': 'PT3M54.72S', 'd...",0.993120,0.984701,1.0,2.082791
647,"[{'mode': 'BIKE', 'duration': 'PT58.14S', 'dep...",1.000000,0.981919,1.0,2.083727


In [36]:
final_df.sort_values(by=["score"], inplace=True)

In [39]:
final_df = final_df.reset_index(drop=True)

In [40]:
final_df

,path,CO2 emissions,time,transfers,score
0,"[{'mode': 'BIKE', 'duration': 'PT47.34S', 'dep...",0.000000,0.002782,0.0,0.002503
1,"[{'mode': 'BIKE', 'duration': 'PT39.24S', 'dep...",0.000000,0.002782,0.0,0.002503
2,"[{'mode': 'BIKE', 'duration': 'PT47.34S', 'dep...",0.000000,0.002782,0.0,0.002503
3,"[{'mode': 'BIKE', 'duration': 'PT39.24S', 'dep...",0.000000,0.002782,0.0,0.002503
4,"[{'mode': 'FOOT', 'duration': 'PT3M9.36S', 'de...",0.007012,0.000000,0.0,0.003506
...,...,...,...,...,...
715,"[{'mode': 'FOOT', 'duration': 'PT3M52.56S', 'd...",0.938315,0.984006,1.0,2.054762
716,"[{'mode': 'BIKE', 'duration': 'PT58.68S', 'dep...",0.993120,0.982615,1.0,2.080913
717,"[{'mode': 'FOOT', 'duration': 'PT3M54.72S', 'd...",0.993120,0.984701,1.0,2.082791
718,"[{'mode': 'BIKE', 'duration': 'PT58.14S', 'dep...",1.000000,0.981919,1.0,2.083727


In [32]:
final_df.iloc[457, 0]

[{'mode': 'BIKE',
  'duration': 'PT47.34S',
  'departure_date': '2023-12-02',
  'departure_time': '18:03',
  'departurePlaceCoordinates': ['6.564732', '46.523904'],
  'arrival_date': '2023-12-02',
  'arrival_time': '18:03',
  'arrivalPlaceCoordinates': [6.566872, 46.522054]},
 {'mode': 'METRO',
  'duration': 'PT5M',
  'departure_date': '2023-12-02',
  'departure_time': '17:33',
  'departurePlace': 'Ecublens VD, EPFL',
  'departurePlaceCoordinates': [6.566144, 46.522197],
  'arrival_date': '2023-12-02',
  'arrival_time': '17:38',
  'arrivalPlaceCoordinates': [6.589803, 46.523267],
  'arrivalPlace': 'Lausanne, Bourdonnette'},
 {'mode': 'BUS',
  'duration': 'PT13M',
  'departure_date': '2023-12-02',
  'departure_time': '17:42',
  'departurePlace': 'Lausanne, Bourdonnette',
  'departurePlaceCoordinates': [6.589803, 46.523267],
  'arrival_date': '2023-12-02',
  'arrival_time': '17:55',
  'arrivalPlaceCoordinates': [6.626641, 46.507473],
  'arrivalPlace': 'Lausanne, Ouchy-Olympique'},
 {'mod

In [33]:
final_df.iloc[289, 0]

[{'mode': 'BIKE',
  'duration': 'PT39.24S',
  'departure_date': '2023-12-02',
  'departure_time': '18:03',
  'departurePlaceCoordinates': ['6.564732', '46.523904'],
  'arrival_date': '2023-12-02',
  'arrival_time': '18:03',
  'arrivalPlaceCoordinates': [6.566144, 46.522197]},
 {'mode': 'METRO',
  'duration': 'PT5M',
  'departure_date': '2023-12-02',
  'departure_time': '17:33',
  'departurePlace': 'Ecublens VD, EPFL',
  'departurePlaceCoordinates': [6.566144, 46.522197],
  'arrival_date': '2023-12-02',
  'arrival_time': '17:38',
  'arrivalPlaceCoordinates': [6.589803, 46.523267],
  'arrivalPlace': 'Lausanne, Bourdonnette'},
 {'mode': 'BUS',
  'duration': 'PT13M',
  'departure_date': '2023-12-02',
  'departure_time': '17:42',
  'departurePlace': 'Lausanne, Bourdonnette',
  'departurePlaceCoordinates': [6.589803, 46.523267],
  'arrival_date': '2023-12-02',
  'arrival_time': '17:55',
  'arrivalPlaceCoordinates': [6.626641, 46.507473],
  'arrivalPlace': 'Lausanne, Ouchy-Olympique'},
 {'mod

In [34]:
final_df

,path,CO2 emissions,time,transfers,score
0,"[{'mode': 'FOOT', 'duration': 'PT54.72S', 'dep...",0.015501,0.018081,0.0,0.024023
1,"[{'mode': 'FOOT', 'duration': 'PT54.72S', 'dep...",0.015501,0.018081,0.0,0.024023
2,"[{'mode': 'FOOT', 'duration': 'PT54.72S', 'dep...",0.877925,0.018081,0.0,0.455235
3,"[{'mode': 'FOOT', 'duration': 'PT54.72S', 'dep...",0.023274,0.015994,1.0,0.726032
4,"[{'mode': 'FOOT', 'duration': 'PT54.72S', 'dep...",0.023274,0.015994,1.0,0.726032
...,...,...,...,...,...
715,"[{'mode': 'BIKE', 'duration': 'PT58.68S', 'dep...",0.022671,0.002086,1.0,0.713213
716,"[{'mode': 'BIKE', 'duration': 'PT58.68S', 'dep...",0.908540,0.002086,1.0,1.156148
717,"[{'mode': 'BIKE', 'duration': 'PT58.68S', 'dep...",0.022671,0.984701,1.0,1.597566
718,"[{'mode': 'BIKE', 'duration': 'PT58.68S', 'dep...",0.022671,0.982615,1.0,1.595689


In [35]:
final_df.drop_duplicates(subset=["path"])

,path,CO2 emissions,time,transfers,score
0,"[{'mode': 'FOOT', 'duration': 'PT54.72S', 'dep...",0.015501,0.018081,0.0,0.024023
1,"[{'mode': 'FOOT', 'duration': 'PT54.72S', 'dep...",0.015501,0.018081,0.0,0.024023
2,"[{'mode': 'FOOT', 'duration': 'PT54.72S', 'dep...",0.877925,0.018081,0.0,0.455235
3,"[{'mode': 'FOOT', 'duration': 'PT54.72S', 'dep...",0.023274,0.015994,1.0,0.726032
4,"[{'mode': 'FOOT', 'duration': 'PT54.72S', 'dep...",0.023274,0.015994,1.0,0.726032
...,...,...,...,...,...
715,"[{'mode': 'BIKE', 'duration': 'PT58.68S', 'dep...",0.022671,0.002086,1.0,0.713213
716,"[{'mode': 'BIKE', 'duration': 'PT58.68S', 'dep...",0.908540,0.002086,1.0,1.156148
717,"[{'mode': 'BIKE', 'duration': 'PT58.68S', 'dep...",0.022671,0.984701,1.0,1.597566
718,"[{'mode': 'BIKE', 'duration': 'PT58.68S', 'dep...",0.022671,0.982615,1.0,1.595689
